In [2]:
import pandas as pd
import re
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
df= pd.read_csv(r"C:\Users\pablo\Documents\GitHub\WhiteHosting\CSVs\CSV Finales\Airbnb Unido.csv")
ids= df["ID"]
coord= df["Localización"]

In [3]:
opts= Options()
opts.add_argument ("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36")
#opts.add_argument("--headless")
#opts.add_argument('--blink-settings=imagesEnabled=false')
driver= webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=opts
)

url= "https://www.google.es/maps/"
lista_cp= []
def strip_cp(direccion):
    pattern= r'\b\d{5}\b'
    match= re.search(pattern, direccion)
    if match:
        return match.group(0)
    else:
        return None

driver.get(url)
sleep(2)
driver.find_element("xpath", '//*[@aria-label="Aceptar todo"]').click()


for x in coord[8000:]:   
    
    driver.find_element("xpath", '//*[@role="combobox"]').send_keys(x)
    driver.find_element("xpath", '//*[@aria-label="Búsqueda"]').click()
    sleep(2.1)
    html= driver.page_source
    soup= bs(html, "lxml")
    direccion= soup.find("span",{"class":"DkEaL"})
    if direccion is None:
        lista_cp.append(None)
    else:
        direccion= direccion.text.strip()
        lista_cp.append(strip_cp(direccion))
    try:
        element= WebDriverWait(driver, 5).until(EC.element_to_be_clickable(("xpath", '//*[@aria-label="Cerrar"]')))
        element.click()
    except:
        pass

driver.close()

In [21]:
airbnb_cp1= pd.DataFrame({"ID": ids[8000:], "CP": lista_cp})
airbnb_cp1.to_csv("airbnb_cp4.csv")

In [22]:
df1=pd.read_csv(r"C:\Users\pablo\Documents\GitHub\WhiteHosting\Pablo\airbnb_cp1.csv")
df2=pd.read_csv(r"C:\Users\pablo\Documents\GitHub\WhiteHosting\Pablo\airbnb_cp2.csv")
df3=pd.read_csv(r"C:\Users\pablo\Documents\GitHub\WhiteHosting\Pablo\airbnb_cp3.csv")
df4=pd.read_csv(r"C:\Users\pablo\Documents\GitHub\WhiteHosting\Pablo\airbnb_cp4.csv")

In [26]:
df_total= pd.concat([df1, df2, df3, df4], ignore_index=True)
df_total=df_total.drop(columns=["Unnamed: 0"])
df_total["CP"]=pd.to_numeric(df_total['CP'], errors='coerce').astype('Int64')

In [37]:
df_total.to_csv("codigos postales airbnb.csv", index=False)